In the [last notebook](1-Build%20Simulations.ipynb), we generated simulated data for five types of parametric tests: a one sample t test, an independent sample t test, a one-way ANOVA with three groups, a one-way ANOVA with eight groups, and a simple linear correlation.

We will calculate the the distribution-based power using equations described in *Statistical Power Analysis for the Social and Behavioral Sciences : Basic and Advanced Techniques* [<a href="#1">1</a>].

This notebook is expected to take about an hour to run. You can limit run time by downloading the precomputed files from [location].

In [1]:
from functools import partial
import os
import pickle
import warnings

import numpy as np
import scipy.stats
import skbio

from machivellian.power import subsample_power
import machivellian.traditional as trad

# Simulation Parameters

In notebook 1, we simulated 100 distributions for each test.

In [2]:
num_rounds = 100

The distributions should be kept in a directory called `simulations` in the current directory. If the downloaded files have been moved to a different location, this directory needs to be adjusted.

In [3]:
sim_location = './simulations/'
if not os.path.exists(sim_location):
    raise ValueError('The simulation directory does not exist.' 
                     'Go back and simulate some data!')

Power calculations are expensive, so users may prefer not to perform them if power has already been calculated.

In [4]:
overwrite = False

We can bootstrap power, so we will calculate power with between 5 and 100 observations, counting by 10s. We will use a critical value of 0.05. Each power calculation will be calculated from 100 tests, and we'll repeat the calcualations 5 times at each depth.

In [5]:
counts = np.arange(5, 100, 10)
alpha = 0.05

num_iter = 100
run_runs = 5

We have five parametric distributions we'll investigate. We'll go through them starting with the categorical tests with increasing group sizes, and then the continous data.

In [6]:
parametric = ['ttest_1', 'ttest_ind', 'anova_3', 'anova_8', 'correlation']
distributions = {}

# 3.1 One Sample T test

A case I t-test checks if an observation is drawn from a sample. We are testing the alternatively hypotheses,

$\begin{matrix}
\textbf{H}_{0} & x = \bar{x} \\
\textbf{H}_{1} & x \neq \bar{x}\\
\end{matrix} \tag{2.1}$

where $\bar{x}$ is the mean of the population, $x$ is the value being compared to the sample, $s$ is the standard devation of the sample, and there are $n$ observations in the sample.

## 3.1.1 Test Statistic

The test statistic for the case I t test is given as
$t = \frac{(\bar{x} - x)\sqrt{n}}{s} \tag{2.2}$
The probability distribution follows a T distribution with $n-1$ degrees of freedom, where $n$ is the number of observations in the sample.


For the emperical test, we will use the `scipy.stats.ttest_1samp` function, which returns a p value.

In [7]:
def emp_ttest_1(sample, x0=0):
    return scipy.stats.ttest_1samp(sample[0], x0)[1]

## 3.1.2 Effect Size

The non centrality paramter for the statistic, $\lambda$ is given by
$\begin{align*}
\lambda &= \frac{t}{\sqrt{n}}\\
&=\frac{1}{\sqrt{n}}\left(\frac{(\bar{x} - x)\sqrt{n}}{s}\right )\\
&=\frac{(\bar{x} - x)}{s}
\end{align*}\tag{2.4}$

We will encorperate this in the power calculation. To allow for testing, the power calculation has been moved into a library.

In [8]:
distributions['ttest_1'] = {'clean_name': 'One Sample T Test',
                            'test': emp_ttest_1,
                            'trad_power': partial(trad.calc_ttest_1, 
                                                  x0=0, 
                                                  counts=counts),
                            'empr_power': partial(subsample_power, 
                                                  counts=counts, 
                                                  num_iter=1000, 
                                                  num_runs=3,
                                                  test=emp_ttest_1)
                            }

# 3.2 Independent T test

The case II t test is a test for two independent samples, to determine if the samples are drawn from different distributions.

$\begin{matrix}
\textbf{H}_{0} & \bar{x}_{1} = \bar{x}_{2} \\
\textbf{H}_{1} & \bar{x}_{1} \neq \bar{x}_{2}\\
\end{matrix} \tag{3.1}$

## 3.2.1 Test Statistic

There are several ways to calculate this t statistic, but we will operate on the assumption that the two populations have different variances, giving the most extensibe calculation of the test statistic. So,

$\begin{align*}
t &= \frac{\bar{x}_{1} - \bar{x}_{2}}{\sqrt{\frac{s_{1}^{2}}{n_{1}} + \frac{s_{2}^{2}}{n_{2}}}}\\
&= \frac{\bar{x}_{1} - \bar{x}_{2}}{\sqrt{\frac{n_{2}s_{1}^{2} + n_{1}s_{2}^{2}}{n_{1}n_{2}}}}
\end{align*}\tag{3.2}$

The t statistic follows a T distribution with $df$ degrees of freedom, where $df$ is given as
$df = \frac{(s_{1}^{2}/n_{1} + s_{2}^{2}/n_{2})^{2}}{(s_{1}^{2}/n_{1})^2/(n_{1}-1) + s_{2}^{2}/n_{2})^2/(n_{2}-1)} \tag{3.3}$

For the sake of simplicity, we'll assume that $n_{1} = n_{2}$, which allows us to redefine equation (2.1) as
$\begin{align*}
t &= \frac{(\bar{x}_{1} - \bar{x}_{2})}{\sqrt{\frac{s_{1}^{2}}{n} + \frac{s_{2}^{2}}{n}}}\\
&= \frac{\sqrt{n}(\bar{x}_{1} - \bar{x}_{2})}{\sqrt{s_{1}^{2} + s_{2}^{2}}}
\end{align*}\tag{3.4}$
which means the test statitic is now drawn from a t distribution with df degrees of freedom, where
df is defined as
$\begin{align*}
df &= \left (n-1 \right ) \left (\frac{\left (s_{1}^{2} + s_{2}^{2}  \right )^{2}}{\left (s_{1}^{2} \right)^{2} + \left (s_{2}^{2}  \right )^{2}} \right )\\
\end{align*}\tag{3.5}$

For the emperical test, we can use the `scipy.stats.ttest_ind` function, which will return a p value.

For the sake of simplicity, we'll assume that $n_{1} = n_{2}$, which allows us to redefine equation (2.1) as
$\begin{align*}
t &= \frac{(\bar{x}_{1} - \bar{x}_{2})}{\sqrt{\frac{s_{1}^{2}}{n} + \frac{s_{2}^{2}}{n}}}\\
&= \frac{\sqrt{n}(\bar{x}_{1} - \bar{x}_{2})}{\sqrt{s_{1}^{2} + s_{2}^{2}}}
\end{align*}\tag{3.4}$
which means the test statitic is now drawn from a t distribution with df degrees of freedom, where
df is defined as
$\begin{align*}
df &= \left (n-1 \right ) \left (\frac{\left (s_{1}^{2} + s_{2}^{2}  \right )^{2}}{\left (s_{1}^{2} \right)^{2} + \left (s_{2}^{2}  \right )^{2}} \right )\\
\end{align*}\tag{3.5}$

For the emperical test, we can use the `scipy.stats.ttest_ind` function, which will return a p value.

In [9]:
def emp_ttest_ind(samples):
    sample1, sample2 = samples
    return scipy.stats.ttest_ind(sample1, sample2)[1]

## 3.2.2 Noncentrality Parameter

The effect size, non-centrality parameter, for an independent sample t test where samples are the same size is once again related to the test statistic as
$\begin{align*}
\lambda &= \frac{t}{\sqrt{n}}\\
&= \left (\frac{\sqrt{n} \left (\bar{x}_{1} - \bar{x}_{2} \right )}{\sqrt{s_{1}^{2} + s_{2}^{2}}} \right ) \left (\frac{1}{\sqrt{n}} \right )\\
&= \left (\frac{\bar{x}_{1}^{2} - \bar{x}_{2}^{2}}{\sqrt{s_{1}^{2} + s_{2}^{2}}} \right )
\end{align*}\tag{3.8}$

In [10]:
distributions['ttest_ind'] = {'clean_name': 'Independent Sample T test',
                              'test': emp_ttest_ind,
                              'trad_power': partial(trad.calc_ttest_ind, 
                                                    counts=counts),
                              'empr_power': partial(subsample_power, 
                                                  counts=counts, 
                                                  num_iter=1000, 
                                                  num_runs=3,
                                                  test=emp_ttest_ind)
                              }

# 3.3 One way Analysis of Variance

Assume there exist a set of samples, $\{S_{1}, S_{2}, ..., S_{k} \}$ where there are a total of $N$ observations distributed over the $k$ groups. The $i$th sample, $S_{i}$ contains $n_{i}$ observations, and has a mean of $\bar{x}_{.i}$ and a standard deviation, $s_{i}$ where

$\begin{align*}
s_{i} = \sqrt{\frac{\sum_{j=1}^{n}{\left (x_{ij} - \bar{x}_{.i} \right)^{2}}}{n_{i}-1}}
\end{align*}\tag{4.1}$

A one-way Analysis of Variance (ANOVA) tests that at least one sample mean in a set of three or more are not equal. Assume that 

$\begin{matrix}
\textbf{H}_{0} & \bar{x}_{1} = \bar{x}_{2} = ... \bar{x}_{k} & \\
\textbf{H}_{1} & \bar{x}_{i} \neq \bar{x}_{j} & \exists i,j \epsilon [1, k], i \neq j
\end{matrix} \tag{4.2}$

## 3.3.1 Test Statistic

Assume there exist a set of samples, $\{S_{1}, S_{2}, ..., S_{k} \}$ where there are a total of $N$ observations distributed over the $k$ groups. The $i$th sample, $S_{i}$ contains $n_{i}$ observations, and has a mean of $\bar{x}_{.i}$ and a standard deviation, $s_{i}$ where

$\begin{align*}
s_{i} = \sqrt{\frac{\sum_{j=1}^{n}{\left (x_{ij} - \bar{x}_{.i} \right)^{2}}}{n_{i}-1}}
\end{align*}\tag{4.1}$

A one-way Analysis of Variance (ANOVA) tests that at least one sample mean in a set of three or more are not equal. Assume that 

$\begin{matrix}
\textbf{H}_{0} & \bar{x}_{1} = \bar{x}_{2} = ... \bar{x}_{k} & \\
\textbf{H}_{1} & \bar{x}_{i} \neq \bar{x}_{j} & \exists i,j \epsilon [1, k], i \neq j
\end{matrix} \tag{4.2}$

The test statistic for ANOVA is given by
$\begin{align*}
F &= \frac{\frac{\textrm{SS}_{\textrm{between}}}{\textrm{DF}_{\textrm{between}}}}{\frac{\textrm{SS}_{\textrm{within}}}{\textrm{DF}_{\textrm{within}}}}
\end{align*}\tag{4.3}$
and test statistic is drawn from an $F$ distribution with $k - 1$ and $N - k$ degrees of freedom [[3](#Zar)].

For the emperical test, we can use the `scipy.stats.f_oneway` function, which will return a p value.

In [11]:
def emp_anova(samples):
    return scipy.stats.f_oneway(*samples)[1]

## 3.3.2 Noncentrality Parameter

Under the alternatively hypothesis, the non-centrality $F'$ is given by

$\begin{align*}
F' = \left(\frac{\textrm{SS}_{\textrm{between}}}{\textrm{SS}_{\textrm{within}}} \right) \left (\frac{\textrm{DF}_{\textrm{within}}}{\textrm{DF}_{\textrm{between}}}{} \right )
\end{align*}\tag{4.9}$

For a given pair of hypotheses, the noncentrality parameter is defined according to equation (2.4), where the grand mean can be substituted for the the test mean. The overall effect size is therefore given as
$\begin{align*}
\lambda &= \sum_{i=1}^{k}{\lambda_{i}^{2}}\\
&= \sum_{i=1}^{k}\left (\frac{\bar{x}_{i} - \bar{x}_{..}}{s_{i}} \right )^{2} 
\end{align*} \tag{4.10}$

Our simulations will focus on the 3 sample ANOVA we generated earlier.

In [12]:
distributions['anova_3'] = {'clean_name': 'ANOVA (3 groups)',
                            'test': emp_anova,
                            'trad_power': partial(trad.calc_anova, 
                                                  counts=counts),
                            'empr_power': partial(subsample_power, 
                                                  counts=counts, 
                                                  num_iter=1000, 
                                                  num_runs=3,
                                                  test=emp_anova)
                              }
distributions['anova_8'] = {'clean_name': 'ANOVA (8 groups)',
                            'test': emp_anova,
                            'trad_power': partial(trad.calc_anova, 
                                                  counts=counts),
                            'empr_power': partial(subsample_power, 
                                                  counts=counts, 
                                                  num_iter=1000, 
                                                  num_runs=3,
                                                  test=emp_anova)
                              }

# 3.4 Pearson's R

Pearson's correlation coeffecient looks for a linear one-to-one relationship between two vectors, $x$ and $y$, both of size $n$. Closely related vectors have a correlation coeffecient with an absloute value of 1, unrelated data have a correlation coeffecient of 0.

## 3.4.1 Test Statistic

The correlation coeffecient between the two vectors is given by
$\begin{align*}
r = \frac{\sum{xy}}{\sqrt{\sum{x^{2}}\sum{y^{2}}}}
\end{align*}\tag{5.1}$

We can test the hypotheses,
$\begin{matrix}
\textbf{H}_{0} & r = 0 \\
\textbf{H}_{1} & x \neq 0\\
\end{matrix} \tag{5.2}$
with a test statistic drawn from the $t$ distribution with $n - 2$ degrees of freedom. The statistic is calculated as
$\begin{align*}
t = \frac{r\sqrt{n-2}}{\sqrt{1 - r^{2}}}
\end{align*}\tag{5.3}$

Scipy's `scipy.stats.pearsonr` can calculate the correlation coeffecient *and* a p value for the coeffecient.

In [13]:
def emp_pearson(samples):
    return scipy.stats.pearsonr(*samples)[1]

## 3.4.2 Noncentrality Parameter

The noncentrality parameter for pearson's correlation coeffecient is given by
$\begin{align}
\lambda = \frac{r}{\sqrt{1 - r^{2}}}
\end{align}\tag{5.4}$

In [14]:
distributions['correlation'] = {'clean_name': "Pearson's R",
                                'test': emp_pearson,
                                'trad_power': partial(trad.calc_pearson, 
                                                      counts=counts),
                                'empr_power': partial(subsample_power, 
                                                      counts=counts, 
                                                      num_iter=100, 
                                                      num_runs=5,
                                                      draw_mode='matched',
                                                      test=emp_pearson)
                                }


# Power Calculations

We will now apply the forumula's we've set up to the final data.

In [15]:
for test_name in parametric:
    
    # Checks the directory location
    sim_dir = os.path.join(sim_location, 'data/%s' % test_name)
    power_dir = os.path.join(sim_location, 'power/%s' % test_name)
    
    # Creates the simulation location if needed
    if not os.path.exists(power_dir):
        os.makedirs(power_dir)
    
    # Draws the distribution and emperical power calculations and the test
    trad_calc = distributions[test_name]['trad_power']
    emp_calc = distributions[test_name]['empr_power']
    test = distributions[test_name]['test']
    
    # Loops through the power calculations
    for i in range(num_rounds):
        sim_fp = os.path.join(sim_dir, 'simulation_%i.p' % i)
        power_fp = os.path.join(power_dir, 'simulation_%i.p' % i)
        # Skips the simulation if the file already exists
        if os.path.exists(power_fp) and not overwrite:
            continue
        
        # Loads the simulated data
        with open(sim_fp, 'rb') as f_:
            simulation = pickle.load(f_)
        samples = simulation['samples']
        traditional = trad_calc(*samples)
        emperical = emp_calc(samples=samples)
        round_summary = {'counts': counts,
                         'emperical': emperical,
                         'traditional': traditional,
                         'num_obs': len(samples[0]),
                         'alpha': alpha,
                         'original_p': test(samples)
                         }
        with open(os.path.join(power_dir, 'simulation_%i.p' % i), 'wb') as f_:
            pickle.dump(round_summary, f_)

We have calculated distribution based and emperical power for five types of parametric distributions. In the [next notebook](), we'll perform the same calculations for distance-based data, we will compare the emperical and distribution based power. We will also evaluate the performance of effect size extrapolation and curve fitting.

# 6. Works Cited

<ol><li>
    <a id="1"></a>Liu, X.S. (2014) *Statistical Power Analysis for the Social and Behavioral Sciences : Basic and Advanced Techniques*. New York: Routledge. 384 pp.
</li></ol>